In [ ]:
alldata = list.files(path = "./6.otu-metabolite-luming-zhizhi/",pattern = ".cor.r.csv$")

kvalue = read.csv('./cnumber.csv')
mmu_kegg_compound <- read.csv('./KEGG_COMPOUND_PATHWAY_mmu.csv')

library(clusterProfiler)

alldata

for (file in alldata){
    outdir = paste0(getwd(),'/8.metabolites.connect.enrichment.zhizhi/',substr(file,1,6))
    if(dir.exists(outdir)){
      print("dir exists")
    }else{
      dir.create(outdir)
    }

    data = read.csv(paste0('./6.otu-metabolite-luming-zhizhi/',file))
    colnames(data)[1] = 'Metabolites'
    df = dplyr::left_join(kvalue,data,by = 'Metabolites')
    df= df[,-1]
    koclp_t = df
    each = list()
    final = list()
    for (i in c(2:length(colnames(koclp_t)))){
        
        each[[i]] = koclp_t[,c(1,i)]
        
        library(dplyr)
        each[[i]]  %>% filter_at(vars(c(2)),all_vars(. !=0)) %>% .$kegg -> final[[i]]
        
    }

    
    disease2gene=mmu_kegg_compound[, c("Pathway", "COMPOUND")]
    disease2name=mmu_kegg_compound[, c("Pathway", "PATHWAY_MAP")]
    
    x = list()
    p = list()

    for (i in c(2:length(each))){

        tryCatch({
        x[[i]] = enricher(final[[i]], TERM2GENE=disease2gene, TERM2NAME=disease2name)
        #write.csv(x[[i]]@result,file = paste0(outdir,'/',colnames(df)[i],'.enrichment.csv'))
            data = x[[i]]@result
            ensembl=strsplit(data$geneID,"/")
            res = list()
            y = list()
            for (j in c(1:length(ensembl))){
                res[[j]] =  kvalue %>%  filter(kegg %in% ensembl[[j]])
                data[j,10] = paste(res[[j]]$Metabolites,collapse = "/")
                colnames(data)[10] = 'name'
            }
        write.csv(data,file = paste0(outdir,'/',colnames(df)[i],'.enrichment.rename.csv'))
            
            library(ggplot2)
            tryCatch({
            p[[i]] = dotplot(x[[i]])
            ggsave(p[[i]],file = paste0(outdir,'/',colnames(df)[i],'.pdf'))
            }, error = function(e) {
            print('chucuop')
            },finally = {
            next})
        

        
        }, error = function(e) {
        print('chucuop')
        },finally = {
        next})
    
    }

    
}